In [4]:
from io import BytesIO
from google.cloud import storage
import librosa as li
import os
import numpy as np

In [ ]:
# the google cloud storage bucket we will be working on
BUCKET_NAME = "speech_emotion_bucket"

# the folder inside the bucket we will be working on
PREFIX = "Raw/"

In [ ]:
# creates a client object to interact with GCS
# loggin in into our GCS environment
client = storage.Client()

# the list of files (blobs) inside the bucket
# stores those files into a variable blobs
blobs = client.list_blobs(BUCKET_NAME, prefix=PREFIX)

In [ ]:
# creates dictionary to store the waveform (y) of each audio file
raw_data = {}

In [ ]:
# the google cloud storage bucket we will be working on
BUCKET_NAME = "speech_emotion_bucket"

# the folder inside the bucket we will be working on
PREFIX = "Raw/"

# creates a client object to interact with GCS
# loggin in into our GCS environment
client = storage.Client()

# the list of files (blobs) inside the bucket
# stores those files into a variable blobs
blobs = client.list_blobs(BUCKET_NAME, prefix=PREFIX)

# creates dictionary to store the waveform (y) of each audio file
raw_data = {}

# iterates trough each file inside the bucket
for blob in blobs:
    if blob.name.endswith(".wav"):

        # downloads each audio file and stores it into a binary variable
        bytes = blob.download_as_bytes(raw_download=True)
        binary = BytesIO(bytes)

        # load the audio into librosa
	    # y is a NumPy array witht he waveform data
        # sr is the sample rate (16khz)
        y, sr = li.load(binary, sr=None)

        # scales the waveform so the maximum value is 1
        y = y / np.abs(y).max()

        # remove silence
        # silence is defined by a decibel threshold (tob_db)
        # default should be around 60 db
        y, _ = li.effects.trim(y)

        # compute the mel spectogram
        # it's a 2d array (rows = frequency ; columns = time ; values = intensity (brightness))
        S = li.feature.melspectrogram(y=y, sr=sr)

        # convert the spectogram from raw energy (power) into decibel scale (log scale)
        S_dB = li.power_to_db(S, ref=np.max)

        # convert the standardized spectogram into an image file & save it in the cloud

        # Stockage en mémoire
        raw_data[blob.name] = {
            “signal”: signal,
            “sampling_rate”: sr
        }
        print(f”{blob.name} → {len(signal)} échantillons à {sr} Hz”)
print(f”\n:white_check_mark: {len(raw_data)} fichiers audio chargés dans `raw_data`.“)

SyntaxError: invalid character '“' (U+201C) (3658399629.py, line 14)